# Topics


In [39]:
import numpy as np
import pandas as pd
import math
import re
import sys
from shapely.geometry import Polygon
from matplotlib import pyplot as plt
from collections import Counter, OrderedDict, namedtuple, defaultdict, ChainMap
from queue import Queue
from copy import deepcopy
import networkx as nx
from functools import cmp_to_key, reduce
from itertools import product, permutations, combinations, combinations_with_replacement
from itertools import repeat
from functools import cache
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import maximum_flow
import json
import time
from z3 import *
from tqdm import tqdm

In [40]:
sys.setrecursionlimit(1500)

In [41]:
with open("15-input", "r") as file:
    lines = file.readlines()
data_raw = "".join(lines)
# data_raw = [line.replace("\n", "") for line in lines]
# data_raw = "\n".join(data_raw)
data_raw

'##################################################\n#O..OOOO......#...O.#O..........OOO..O#.O..OO..###\n#...O#...#...O.O...O.O........#OO...O.#O..O.O..O.#\n#..O.......#O..OO.#..O..O...O....O#O.O.....O.....#\n#O...O.O...O.O#.O..O....O..#OOO......#O......#.O.#\n#O........O.#.#....OO............OO...O..O.......#\n#..O.OO.OOOO.O.....OO....#OO....O.........#...#..#\n#.....O........O....#....#OOO.O..................#\n#...O...#O#.O.......O.O.OO.##...O.OO.O.....O...#.#\n#OO.O#O....O..O....#O..OO....O.O..#OO.#O....OO.#.#\n#O......#..O..OO#..OOO...O.#OOO...O......O.O..O..#\n#.#....O.........OO.OO......O#O#..O#O......O...O.#\n#O.#....O.OOO.#O.......O....O.OO...#..#...O......#\n#O.....OOO.#....O.OO.#O.O................O.OO...##\n#..O.#......O#OO...#...O.....O##..O..#..O...OO...#\n##.O.O#.O.OO..O.....OO...##.....O.O.......#O...###\n##..#......O.O#.O.O.O....O#OOO...O.#O...........O#\n#O..........O...........O...O.#......O..O..O.O#O.#\n#.......O#.O.O.O....O.....O.OO..O.OO....O.#..O.OO#\n#O...OO.#..

In [45]:
test_data_raw = r"""##########
#..O..O.O#
#......O.#
#.OO..O.O#
#..O@..O.#
#O#..O...#
#O..O..O.#
#.OO.O.OO#
#....O...#
##########

<vv>^<v^>v>^vv^v>v<>v^v<v<^vv<<<^><<><>>v<vvv<>^v^>^<<<><<v<<<v^vv^v>^
vvv<<^>^v^^><<>>><>^<<><^vv^^<>vvv<>><^^v>^>vv<>v<<<<v<^v>^<^^>>>^<v<v
><>vv>v^v^<>><>>>><^^>vv>v<^^^>>v^v^<^^>v^^>v^<^v>v<>>v^v^<v>v^^<^^vv<
<<v<^>>^^^^>>>v^<>vvv^><v<<<>^^^vv^<vvv>^>v<^^^^v<>^>vvvv><>>v^<<^^^^^
^><^><>>><>^^<<^^v>>><^<v>^<vv>>v>>>^v><>^v><<<<v>>v<v<v>vvv>^<><<>^><
^>><>^v<><^vvv<^^<><v<<<<<><^v<<<><<<^^<v<^^^><^>>^<v^><<<^>>^v<v^v<v^
>^>>^v>vv>^<<^v<>><<><<v<<v><>v<^vv<<<>^^v^>^^>>><<^v>>v^v><^^>>^<>vv^
<><^^>^^^<><vvvvv^v<v<<>^v<v>v<<^><<><<><<<^^<<<^<<>><<><^^^>^^<>^>v<>
^^>vv<^v^v<vv>^<><v<^v>^^^>>>^^vvv^>vvv<>>>^<^>>>>>^<<^v>^vvv<>^<><<v>
v^^>>><<^^<>>^v^<v^vv<>v^<<>^<^v^v><^<<<><<^<v><v<>vv>>v><v^<vv<>v^<<^"""

test_data_raw2 = r"""########
#..O.O.#
##@.O..#
#...O..#
#.#.O..#
#...O..#
#......#
########

<^^>>>vv<v>>v<<"""

test_data_raw3 = r"""#######
#...#.#
#.....#
#..OO@#
#..O..#
#.....#
#######

<vv<<^^<<^^"""


Robot = namedtuple("Robot", ["px", "py","vx", "vy"])

def preprocess_data (data):
    
    field, moves = data.split("\n\n")    

    rows = [list(row) for row in field.split("\n")]
    return np.array(rows, dtype='U10'), moves.replace("\n", "")



test_data = preprocess_data(test_data_raw)
test_data2 = preprocess_data(test_data_raw2)
test_data3 = preprocess_data(test_data_raw3)
display(test_data)


(array([['#', '#', '#', '#', '#', '#', '#', '#', '#', '#'],
        ['#', '.', '.', 'O', '.', '.', 'O', '.', 'O', '#'],
        ['#', '.', '.', '.', '.', '.', '.', 'O', '.', '#'],
        ['#', '.', 'O', 'O', '.', '.', 'O', '.', 'O', '#'],
        ['#', '.', '.', 'O', '@', '.', '.', 'O', '.', '#'],
        ['#', 'O', '#', '.', '.', 'O', '.', '.', '.', '#'],
        ['#', 'O', '.', '.', 'O', '.', '.', 'O', '.', '#'],
        ['#', '.', 'O', 'O', '.', 'O', '.', 'O', 'O', '#'],
        ['#', '.', '.', '.', '.', 'O', '.', '.', '.', '#'],
        ['#', '#', '#', '#', '#', '#', '#', '#', '#', '#']], dtype='<U10'),
 '<vv>^<v^>v>^vv^v>v<>v^v<v<^vv<<<^><<><>>v<vvv<>^v^>^<<<><<v<<<v^vv^v>^vvv<<^>^v^^><<>>><>^<<><^vv^^<>vvv<>><^^v>^>vv<>v<<<<v<^v>^<^^>>>^<v<v><>vv>v^v^<>><>>>><^^>vv>v<^^^>>v^v^<^^>v^^>v^<^v>v<>>v^v^<v>v^^<^^vv<<<v<^>>^^^^>>>v^<>vvv^><v<<<>^^^vv^<vvv>^>v<^^^^v<>^>vvvv><>>v^<<^^^^^^><^><>>><>^^<<^^v>>><^<v>^<vv>>v>>>^v><>^v><<<<v>>v<v<v>vvv>^<><<>^><^>><>^v<><^vvv<^^<><v<<<<<><^v<<

In [46]:
full_data = preprocess_data(data_raw)
full_data

(array([['#', '#', '#', ..., '#', '#', '#'],
        ['#', 'O', '.', ..., '#', '#', '#'],
        ['#', '.', '.', ..., 'O', '.', '#'],
        ...,
        ['#', '.', '.', ..., '#', '.', '#'],
        ['#', '.', '.', ..., 'O', '.', '#'],
        ['#', '#', '#', ..., '#', '#', '#']], dtype='<U10'),
 '^^^>^v<<>><>vv<><v^>>>^vv>^v>^>>>^<v>>^<^>^^>v>>>v<^>v<^vv>vvv^<<^<>v^^^<^^<>^<>^v^>v<<vv><v<<><v^<^<v<<v^vv<><v><^v><^v<^>>vv>>><v^><^>^^>v<v<<v^>^>^vv<^<^>^^v^<>v>v<^vv<>vv><>>v>^<v>v<^^^>v^^^<<vv<^^v<v^>v<^<<^>><>^v<v>><<<<v>v<v^<^v><vv<v>^<><^v>^^^v><<>vv>v<<<^v<^v<<>><^^v>^v<^^<><<v>^>><vv>v^vv><<v^^^<>>v^<<<v>><>><<<^><vv>^v>>>><vv<vv<>^^^<v^^^<v>v<<<v>^<><v^><><^><v<<<>^<>>^vv^<^<>><>^<<v>v<vv>><<>^vv^v<<><<v<^><v^vv^^<^>^vv<><v^^vv>^^><><>>^^<^>^^v>v>^v<^v^<^<^>v<v^^v>>v^<v^^^>vvvvv<v<^v<><<<vvv<<<>>^^^v>^>>v>><><>>>>>>vv^^<>v<<v<<^<><><>>>v^^<<<>vv>v<v^^<>^^<^<v<<v<<^><>>^>vv^v<><v<v<v>>v>^^vvv<v>v^^v<vv<^v^^<>>v^<v<<>v<<<><^vv<^^><>^v^>><^^v^^^v<^>vv<^v^>^<v<<v<<>^vv><^<<<<v><<<>^

In [47]:

def solution (data, verbose=False):
    field = data[0]
    moves = data[1]

    def display_field(field):
        field_list = field.tolist()
        field_str = "\n".join(["".join(row)   for row in field_list])
        return field_str

    def tuple_add (t1, t2):
        return tuple(map(sum, zip(t1, t2)))

    def move_in_dir(position, command):
        if command == '^':
            return tuple_add(position, (-1,0))
        elif command == 'v':
            return tuple_add(position, (1,0))
        elif command == '<':
            return tuple_add(position, (0,-1))
        elif command == '>':
            return tuple_add(position, (0,1))
        else:
            raise ValueError("Unknown Command")
    
    field_final = field.copy()
    player_pos = tuple(np.argwhere(field == '@')[0])

    for i, cmd in enumerate(list(moves)):
        # print(display_field(field_final))
        # print("\n")
        # print(f"After move {i}: '{cmd}'")
        
        
        next_position = move_in_dir(player_pos, cmd)
        if field_final[next_position] == '#':
            continue
        if field_final[next_position] == '.':
            field_final[next_position] = '@'
            field_final[player_pos] = '.'
            player_pos = next_position
        else:
            trace_position = move_in_dir(next_position, cmd)

            while field_final[trace_position] == 'O':
                trace_position = move_in_dir(trace_position, cmd)
                
            if field_final[trace_position] == '.':
                field_final[trace_position] = 'O'
                field_final[next_position] = '@'
                field_final[player_pos] = '.'
                player_pos = next_position

    print(display_field(field_final))
    return sum([100 * pos[0] + pos[1] for pos in np.argwhere(field_final == 'O').tolist()])



sol = solution(test_data, verbose=True)
print(sol)

sol = solution(test_data2, verbose=True)
print(sol)



##########
#.O.O.OOO#
#........#
#OO......#
#OO@.....#
#O#.....O#
#O.....OO#
#O.....OO#
#OO....OO#
##########
10092
#######
#@..#.#
#.O...#
#..O..#
#..O..#
#.....#
#######
908


In [34]:
sol = solution(full_data)
print(sol)

##################################################
#OO.......OOOO#OO.OO#OO.O....O.OOOO..O#OO..OOOO###
#O...#...#OOO............OO...#.O..OO.#.....O...O#
#O.......OO#O....O#......O......OO#O.O...........#
#O.........OOO#............#.....O..O#O......#..O#
#O.........O#O#.....O............................#
#O.........O............O#O..O.......O....#...#..#
#.....O.....O.....OO#.O..#OO..............O......#
#.......#.#........OOO.O...##.........OOOOO.O..#.#
#O...#...O......O..#OOO..OO....O..#O@.#O.......#.#
#.O.....#.......#OOOOO....O#OOOOO...O...O..O..O..#
#O#O....OO...O.O...O.....OOOO#O#...#.O..O....O...#
#O.#.........O#........OO...O.OOOO.#..#.OO......O#
#O.........#.OOOO..O.#.O......O...O.O...O.......##
#O.O.#.....O.#O...O#..........##..O..#..O.O.O..OO#
##O.OO#O..........OOOO...##..O.O..........#O...###
##..#OO...O..O#O..O...O.OO#.OO.....#O..........OO#
#OO...OOOOO..O.......OOOOOO..O#......OO.O.....#O.#
#O.....OO#.OOO.......OOO.....OOO......O.O.#...OOO#
#......O#O......#.............O

# Part 2

In [ ]:

def solution2 (data, verbose=False):
    def update_field(f):
        new_field = np.full((f.shape[0], f.shape[1]* 2) , '.')
        for x in range(f.shape[0]):
            for y in range(f.shape[1]):
                if f[(x,y)] == '#':
                    new_field[(x, y*2)] = '#'
                    new_field[(x, y*2 +1)] = '#'
                elif f[(x,y)] == 'O':
                    new_field[(x, y*2)] = '['
                    new_field[(x, y*2 +1)] = ']'
                elif f[(x,y)] == '@':
                    new_field[(x, y*2)] = '@'
                    new_field[(x, y*2 +1)] = '.'
        return new_field

    field = update_field(data[0])
    moves = data[1]

    def display_field(field):
        field_list = field.tolist()
        field_str = "\n".join(["".join(row)   for row in field_list])
        return field_str

    def tuple_add (t1, t2):
        return tuple(map(sum, zip(t1, t2)))

    def move_in_dir(position, command):
        if command == '^':
            return tuple_add(position, (-1,0))
        elif command == 'v':
            return tuple_add(position, (1,0))
        elif command == '<':
            return tuple_add(position, (0,-1))
        elif command == '>':
            return tuple_add(position, (0,1))
        else:
            raise ValueError("Unknown Command")
    
    field_final = field.copy()
    player_pos = tuple(np.argwhere(field == '@')[0])

    field_saves = []
    fields_to_show = []
    for i, cmd in enumerate(list(moves)):
        field_saves.append(field_final.copy())
        
        next_position = move_in_dir(player_pos, cmd)
        if field_final[next_position] == '#':
            continue
        if field_final[next_position] == '.':
            field_final[next_position] = '@'
            field_final[player_pos] = '.'
            player_pos = next_position
        else:
            # left, right
            if cmd in [">", "<"]:
                trace_position = move_in_dir(next_position, cmd)
                swaps = [( next_position, '@'), [trace_position, field_final[next_position]]]
                while field_final[trace_position] in ["[", "]"]:
                    last_trace = trace_position
                    trace_position = move_in_dir(trace_position, cmd)
                    swaps.append(( trace_position, field_final[last_trace]))
                
                
                if field_final[trace_position] == '.':
                    for swap in swaps:
                        field_final[swap[0]] = swap[1]

                    field_final[player_pos] = '.'
                    player_pos = next_position
            # up, down
            else:
                level = player_pos[0]
                next_level = next_position[0]
                boxes = OrderedDict({level: [player_pos[1]]})
                above = [field_final[(next_level ,box_y)] for box_y in boxes[player_pos[0]]]
                while '#' not in above:
                    boxes[next_level] = []
                    for box_y in boxes[level]:
                        if field_final[(next_level, box_y)] == '[':
                            boxes[next_level].append(box_y) 
                            boxes[next_level].append(box_y+1)
                        elif field_final[(next_level, box_y)] == ']':
                            boxes[next_level].append(box_y-1)
                            boxes[next_level].append(box_y)
                    boxes[next_level] = list(set(boxes[next_level]))
                    level = next_level

                    
                        
                    next_level = move_in_dir((level, 0), cmd)[0]
                    above = [field_final[(next_level ,box_y)] for box_y in boxes[level]]
                    if len(set(above)) == 1 and '.' in above:
                        # move forward
                        for l in reversed(list(boxes.keys())):
                            l_next = move_in_dir((l,0), cmd)[0]
                            for box_y in boxes[l]:
                                field_final[(l_next, box_y)] = field_final[(l, box_y)] 
                                field_final[(l, box_y)] = '.'

                        field_final[next_position] = '@'
                        field_final[player_pos] = '.'
                        player_pos = next_position
                        break
                if len(list(boxes.keys())) > 2:
                    fields_to_show.append(i)            


    field_saves.append(field_final.copy())

    # for index in fields_to_show:
        
    #     print(display_field(field_saves[index]))
    #     print(f"After move {index}: '{list(moves[index])}'")
    #     print(display_field(field_saves[index+1]))
    #     print("\n")

    print(display_field(field_final))

    # calc solution from final field
    total = 0
    for box_left in np.argwhere(field_final == '[').tolist():
        box_right = [box_left[0], box_left[1] + 1]
        total += 100*box_left[0] + box_left[1]

    return total



sol = solution2(test_data, verbose=True)
print(sol)

# sol = solution2(test_data3, verbose=True)
# print(sol)

####################
##[].......[].[][]##
##[]...........[].##
##[]........[][][]##
##[]......[]....[]##
##..##......[]....##
##..[]............##
##..@......[].[][]##
##......[][]..[]..##
####################
9021


In [86]:
sol = solution2(full_data)
print("Solution", sol)

####################################################################################################
##[]....[][][][]............##[][]..[][]##[][].....[][].......[][][][][][][]##..[]....[][]....######
##......[]##......##......[]..[][][]..[]....................##..[]......[]..##[]....[][][]....[]..##
##....[]..............##[]..[].[]...##[]...[]..[]............[].....##..[][][][]......[]..........##
##[]......[]..[]......[]..[]##[]......................##...[].............##[]............##..[]..##
##[]................[]..##[]##[]..........................[][].................[].................##
##....[]..[][]..[][][][][].[].[]...........[].....##[]..[].[].[]................[][]##....[]##....##
##..........[]..........................##........##[][][].[].......[].........[][][].............##
##......[]......##[]##............................[]..####.[]...[]..[]..........[]..[]........##..##
##[][]..[]##[].....[].................##......[]...[][][]...........##......##....[][].....